# Project 3

## Getting Insights into Images and their Metadata


Analysis will include:
...

In [41]:
%matplotlib inline
import scipy
import scipy.io
import numpy as np
import torch
import torchvision
from torchvision import models, transforms
import sqlite3
import matplotlib
from matplotlib import pyplot as plt
from PIL import Image

In [42]:
vgg16 = torchvision.models.vgg16(pretrained=True) # very slow command

/Users/katy/anaconda3/envs/mlds/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/katy/anaconda3/envs/mlds/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


funny, for me that command took maybe half a second? /sara

In [43]:
# vgg16.features is a series of modules.
# we can chain an input through the modules via the forward method.
phi = vgg16.features.forward

def preprocess_image(img):

    # commented out lines are (probably) unnecessary? I hope?
    transform = transforms.Compose([
        #transforms.Resize(256), # or 224
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    batch = transform(img)
    # batch = torch.unsqueeze(batch, 0)
    return batch 

def fetch_img(img_index:int, healthy:bool=True):
    if healthy:
        return Image.open(f"Apple___healthy/image ({img_index}).JPG")
    else:
        return Image.open(f"Apple___Black_rot/image ({img_index}).JPG")

def img_features(img_index:int, healthy:bool=True, flatten=False):
    res = phi(preprocess_image(fetch_img(img_index, healthy=healthy)))
    if flatten:
        res = torch.flatten(res)
    return res

phi_out_shape = img_features(1, healthy=True).shape
phi_out_shape


torch.Size([512, 8, 8])

The guessing process consists of the following pipeline of functions:

fetch_img -> preprocess_image -> phi (aka our neural network) -> flatten -> guess

where guess requires w to be calculated, which is the normalized flattened difference of mean phi outputs of healthy and unhealthy images.


In [44]:
# first we need to calculate the means for both healthy and unhealthy image features
def mean_features(indices, healthy=True, flatten = True):
    mean = None
    for x in indices:
        if mean is None:
            mean = img_features(x, healthy=healthy, flatten=flatten)
        else:
            mean += img_features(x, healthy=healthy, flatten=flatten)

    return mean / len(indices)

training_indices = range(1, 20) # first few images as training set. Can always be changed later.

mean_healthy = mean_features(training_indices, healthy=True)
mean_rotted = mean_features(training_indices, healthy=False)

w = mean_rotted - mean_healthy
w = w / torch.linalg.norm(w) 

def guess(x): # takes a flattened tensor of image features
    return torch.dot(w, x) # magnitude of projection onto line from mean to mean

def img_guess(img_index, healthy=True): 
    return guess(img_features(img_index, healthy, flatten=True))

We want to test the guessing with some examples:

In [45]:
# see if we can correctly identify the first healthy/unhealthy leaves not in the training set
test_healthy = img_guess(21, healthy=True)
test_rotted = img_guess(21, healthy=False)

mean_healthy_position = guess(mean_healthy)
mean_rotted_position = guess(mean_rotted)
# smaller values imply higher confidence
true_neg = torch.abs(test_healthy - mean_healthy_position)
true_pos = torch.abs(test_rotted - mean_rotted_position)
false_neg = torch.abs(test_rotted - mean_healthy_position)
false_pos = torch.abs(test_healthy - mean_rotted_position)
print(true_neg)
print(true_pos)
print(false_neg)
print(false_pos)
# smaller values imply higher confidence

tensor(4.8400, grad_fn=<AbsBackward0>)
tensor(2.5386, grad_fn=<AbsBackward0>)
tensor(34.9783, grad_fn=<AbsBackward0>)
tensor(32.6769, grad_fn=<AbsBackward0>)


interpretation: Our model can quite accurately determine if the image is rotted based on just 20 images of training data.